In [1]:
import pandas as pd
import numpy as np
import math
import time
import json
import copy
import os
import sys
sys.path.append('../..')

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Subset


from fedrpdp.datasets.fed_heart_disease import (
    BaselineModel,
    BaselineLoss,
    FedHeartDisease,
    metric,
)

from fedrpdp.utils.rpdp_utils import (
    get_sample_rate_curve,
    MultiLevels, 
    MixGauss, 
    Pareto,
)

device = "cuda:0"
lr = 0.1

train_data = FedHeartDisease(train=True, pooled=True)
test_data = FedHeartDisease(train=False, pooled=True)
train_loader = DataLoader(
    train_data,
    batch_size=len(train_data),
    shuffle=False,
    num_workers=0,
)
test_loader = DataLoader(
    test_data,
    batch_size=len(test_data),
    shuffle=False,
    num_workers=0,
)

model_init = BaselineModel().to(device)
torch.manual_seed(42) 

noise_multiplier = 10.0
max_grad_norm = 1.0
max_epochs = 100
delta = 1e-3

total_points = len(train_data)
num_level1 = int(total_points * 0.7)
num_level2 = int(total_points * 0.2)
num_level3 = total_points - num_level1 - num_level2

def train(model, device, train_loader, optimizer, criterion, metric, running_norms=None):
    model.train()
    data, target = next(iter(train_loader))
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)

    # compute train acc
    correct = metric(target.detach().cpu().numpy(), output.detach().cpu().numpy())
    train_acc = correct / len(target)
    
    # compute train loss
    loss = criterion(output, target)
    train_loss = loss.item()
    loss.backward()

    if running_norms is not None:
        gradient_norms = optimizer.step(running_norms)
        gradient_norms_sq = gradient_norms * gradient_norms
        return train_loss, train_acc, gradient_norms_sq
    
    else:
        optimizer.step()
        return train_loss, train_acc
    

def test(model, device, test_loader, criterion, metric):
    model.eval()
    with torch.no_grad():
        data, target = next(iter(test_loader))
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss = criterion(output, target).item()
        
        correct = metric(target.detach().cpu().numpy(), output.detach().cpu().numpy())
        test_acc = 1. * correct / len(target)

    return test_loss, test_acc

# GD with RDP Filter (NeurIPS'21)

In [2]:
from torchdp import PrivacyEngine
from fedrpdp.accountants.utils import get_noise_multiplier

def generate_rdp_orders():
    dense = 1.07
    alpha_list = [int(dense ** i + 1) for i in range(int(math.floor(math.log(1000, dense))) + 1)]
    alpha_list = np.unique(alpha_list)
    return alpha_list

norm_sq_budgets = [5] * num_level1 + [20] * num_level2 + [100] * num_level3

_model = copy.deepcopy(model_init)
_train_loader = copy.deepcopy(train_loader)
optimizer = optim.SGD(_model.parameters(), lr=lr, momentum=0)
criterion = BaselineLoss()
privacy_engine = PrivacyEngine(
    module=_model,
    batch_size=total_points,
    sample_size=total_points,
    alphas=generate_rdp_orders(),
    noise_multiplier=noise_multiplier,
    max_grad_norm=max_grad_norm,
    norm_sq_budget=norm_sq_budgets,
    should_clip=True,
)
privacy_engine.attach(optimizer)

In [3]:
running_grad_sq_norms = [ torch.Tensor([0] * total_points).to(device) ]
results_all_reps = [
    {
        "test_loss": 0, 
        "test_acc": 0, 
        "seconds": 0, 
        "num_active_points": total_points,
        "norm_sq_budgets": set(norm_sq_budgets), 
        "e": json.dumps({0: 0, num_level1:0, num_level1+num_level2:0}), 
        "d": delta, 
        "nm": round(noise_multiplier, 2), 
        "norm": max_grad_norm
    }
]

for epoch in range(1, max_epochs + 51):
    # compute activate points
    temp = running_grad_sq_norms[-1].cpu().numpy()
    num_active_points = np.sum(np.round(temp, 4) < np.array(norm_sq_budgets))

    start = time.time()
    train_loss, train_acc, grad_sq_norms = train(_model, device, _train_loader, optimizer, criterion, metric, running_grad_sq_norms[-1])
    end = time.time()
    seconds = end - start
    running_grad_sq_norms.append(running_grad_sq_norms[-1] + grad_sq_norms)
    
    epsilon1 = privacy_engine.get_epsilon(privacy_engine.norm_sq_budget[0], delta)[0]
    epsilon2 = privacy_engine.get_epsilon(privacy_engine.norm_sq_budget[num_level1], delta)[0]
    epsilon3 = privacy_engine.get_epsilon(privacy_engine.norm_sq_budget[num_level1 + num_level2], delta)[0]
    
    temp = running_grad_sq_norms[-1].cpu().numpy()
    print(f"Epoch: {epoch}: seconds = {seconds}")
    print(
        f"Train Loss: {train_loss:.4f} \t Acc: {train_acc:.4f}"
        f"| δ: {delta} ε1 = {epsilon1:.4f} ε2 = {epsilon2:.4f} ε3 = {epsilon3:.4f}."
    )
    
    test_loss, test_acc = test(_model, device, test_loader, criterion, metric)
    print(
        f"Test  Loss: {test_loss:.4f} \t Acc: {test_acc:.4f}\n"
    )

    results_all_reps.append(
        {
            "test_loss": round(test_loss,4), 
            "test_acc": round(test_acc,4), 
            "seconds": round(seconds,4), 
            "num_active_points": num_active_points.item(),
            "norm_sq_budgets": set(running_grad_sq_norms[-1].cpu().numpy()),
            "e": json.dumps({0:epsilon1, num_level1:epsilon2, num_level1+num_level2:epsilon3}), 
            "d": delta, 
            "nm": round(noise_multiplier,2), 
            "norm": max_grad_norm
        }
    )

    results = pd.DataFrame.from_dict(results_all_reps)
    results.to_csv("results_filter.csv", index=False)
    
    if num_active_points < 10:
        break
    epoch += 1

Epoch: 1: seconds = 0.6004955768585205
Train Loss: 0.6580 	 Acc: 0.6173% | δ: 0.001 ε1 = 0.3768 ε2 = 0.3768 ε3 = 0.3768.
Test  Loss: 0.6676 	 Acc: 0.5748

Epoch: 2: seconds = 0.008775472640991211
Train Loss: 0.6435 	 Acc: 0.6276% | δ: 0.001 ε1 = 0.5358 ε2 = 0.5358 ε3 = 0.5358.
Test  Loss: 0.6536 	 Acc: 0.5984

Epoch: 3: seconds = 0.008673906326293945
Train Loss: 0.6302 	 Acc: 0.6543% | δ: 0.001 ε1 = 0.6589 ε2 = 0.6589 ε3 = 0.6589.
Test  Loss: 0.6409 	 Acc: 0.6063

Epoch: 4: seconds = 0.008498907089233398
Train Loss: 0.6182 	 Acc: 0.6626% | δ: 0.001 ε1 = 0.7636 ε2 = 0.7636 ε3 = 0.7636.
Test  Loss: 0.6303 	 Acc: 0.6220

Epoch: 5: seconds = 0.008757829666137695
Train Loss: 0.6076 	 Acc: 0.6790% | δ: 0.001 ε1 = 0.8563 ε2 = 0.8563 ε3 = 0.8563.
Test  Loss: 0.6181 	 Acc: 0.6378

Epoch: 6: seconds = 0.008407354354858398
Train Loss: 0.5962 	 Acc: 0.6996% | δ: 0.001 ε1 = 0.8563 ε2 = 0.9417 ε3 = 0.9417.
Test  Loss: 0.6133 	 Acc: 0.6457

Epoch: 7: seconds = 0.008812904357910156
Train Loss: 0.5918 

Epoch: 54: seconds = 0.008536338806152344
Train Loss: 0.5134 	 Acc: 0.7634% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 3.0016.
Test  Loss: 0.5284 	 Acc: 0.7402

Epoch: 55: seconds = 0.008139371871948242
Train Loss: 0.5132 	 Acc: 0.7613% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 3.0316.
Test  Loss: 0.5278 	 Acc: 0.7480

Epoch: 56: seconds = 0.008657693862915039
Train Loss: 0.5129 	 Acc: 0.7654% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 3.0616.
Test  Loss: 0.5268 	 Acc: 0.7441

Epoch: 57: seconds = 0.008078575134277344
Train Loss: 0.5123 	 Acc: 0.7654% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 3.0916.
Test  Loss: 0.5250 	 Acc: 0.7520

Epoch: 58: seconds = 0.008150100708007812
Train Loss: 0.5113 	 Acc: 0.7675% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 3.1216.
Test  Loss: 0.5255 	 Acc: 0.7402

Epoch: 59: seconds = 0.008104085922241211
Train Loss: 0.5121 	 Acc: 0.7572% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 3.1516.
Test  Loss: 0.5244 	 Acc: 0.7323

Epoch: 60: seconds = 0.008177995681762695
Train Loss

Test  Loss: 0.5012 	 Acc: 0.7638

Epoch: 112: seconds = 0.00844883918762207
Train Loss: 0.4908 	 Acc: 0.7654% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 4.2269.
Test  Loss: 0.5014 	 Acc: 0.7638

Epoch: 113: seconds = 0.00823068618774414
Train Loss: 0.4908 	 Acc: 0.7654% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 4.2269.
Test  Loss: 0.5010 	 Acc: 0.7677

Epoch: 114: seconds = 0.008767843246459961
Train Loss: 0.4900 	 Acc: 0.7675% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 4.2269.
Test  Loss: 0.5008 	 Acc: 0.7677

Epoch: 115: seconds = 0.008516311645507812
Train Loss: 0.4897 	 Acc: 0.7675% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 4.2269.
Test  Loss: 0.5007 	 Acc: 0.7677

Epoch: 116: seconds = 0.008466005325317383
Train Loss: 0.4897 	 Acc: 0.7675% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 4.2269.
Test  Loss: 0.5012 	 Acc: 0.7677

Epoch: 117: seconds = 0.008597612380981445
Train Loss: 0.4897 	 Acc: 0.7695% | δ: 0.001 ε1 = 0.8563 ε2 = 1.7635 ε3 = 4.2269.
Test  Loss: 0.5015 	 Acc: 0.7638

Epoch: 118: se

# Results

In [4]:
# results = pd.read_csv("results_filter.csv")
# np.array(results["seconds"].tolist()).mean()

In [5]:
# # Plot
# import numpy as np
# import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_style('whitegrid', {'axes.linewidth': 1, 'axes.edgecolor':'black'}) #风格、轮廓线
# legend_font = {'style': 'normal', 'size': 8, 'weight': "normal"}
# label_font = {'family':'sans-serif', 'size': 10.5, 'weight': "normal"}
# title_font = {'family':'sans-serif', 'size': 10.5, 'weight': "bold"}

# # privacy_engine.steps = np.floor(privacy_engine.norm_sq_budget[0]/max_grad_norm**2)
# # epsilon1 = privacy_engine.get_epsilon(privacy_engine.norm_sq_budget[0], delta)[0]
# # privacy_engine.steps = np.floor(privacy_engine.norm_sq_budget[num_level1]/max_grad_norm**2)
# # epsilon2 = privacy_engine.get_epsilon(privacy_engine.norm_sq_budget[num_level1], delta)[0]
# # privacy_engine.steps = np.floor(privacy_engine.norm_sq_budget[num_level1 + num_level2]/max_grad_norm**2)
# # epsilon3 = privacy_engine.get_epsilon(privacy_engine.norm_sq_budget[num_level1 + num_level2], delta)[0]

# active_points = np.array(results["num_active_points"].tolist())
# print(
#     f"δ: {delta} ε1 = {epsilon1:.2f} ε2 = {epsilon2:.2f} ε3 = {epsilon3:.2f}."
# )
# print(active_points)
# print(np.floor(norm_sq_budgets[0]/(max_grad_norm**2)), np.floor(norm_sq_budgets[num_level1]/(max_grad_norm**2)), np.floor(norm_sq_budgets[num_level1 + num_level2]/(max_grad_norm**2) ))

# fig, ax = plt.subplots(1, 1, figsize=(3, 3), constrained_layout=True, dpi=500)
# sns.lineplot(data=active_points)
# ax.set_xlabel("Step", **label_font)
# ax.set_ylabel(f"Number of active points", **label_font)
# ax.set_ylim(0, 500)
# ax.set_title("ThreeLevels", **title_font)
# # f"ThreeLevels\n ε = {epsilon1:.2f}(70%) / {epsilon2:.2f}(20%) / {epsilon3:.2f}(10%), δ = {delta}"

# plt.axvline(np.floor(norm_sq_budgets[0]/(max_grad_norm**2))+1, 0, 1, c='m', linewidth = 1)
# plt.axvline(np.floor(norm_sq_budgets[num_level1]/(max_grad_norm**2))+1, 0, 1, c='m', linewidth = 1)
# plt.axvline(np.floor(norm_sq_budgets[num_level1 + num_level2]/(max_grad_norm**2))+1, 0, 1, c='m', linewidth = 1)

# plt.savefig('./filter_active_points_heart_disease', bbox_inches='tight')



# GD with SCF

In [6]:
from fedrpdp.accountants.utils import get_noise_multiplier
from fedrpdp import PrivacyEngine

curve_fn = get_sample_rate_curve(
    target_delta = delta,
    noise_multiplier = noise_multiplier,
    num_updates = max_epochs,
    num_rounds = None,
    client_rate = None
)
epsilon_budgets = [epsilon1] * num_level1 + [epsilon2] * num_level2 + [epsilon3] * num_level3

_model = copy.deepcopy(model_init)
_train_loader = copy.deepcopy(train_loader)
optimizer = optim.SGD(_model.parameters(), lr=lr, momentum=0)
criterion = BaselineLoss()

privacy_engine = PrivacyEngine(accountant="pers_rdp", noise_multiplier=noise_multiplier)
privacy_engine.sample_rate_fn = curve_fn
per_sample_rate = [float(privacy_engine.sample_rate_fn(x)) for x in epsilon_budgets]
print(round(min(epsilon_budgets),4), round(min(per_sample_rate),4))
print(round(max(epsilon_budgets),4), round(max(per_sample_rate),4))
if max(per_sample_rate) == 0.0:
    raise ValueError("Hyper parameter errors! The maximum value of per_sample_rates is zero!")
privacy_engine.sample_rate = per_sample_rate # TODO: make it as an internal func of PrivacyEngine
print(set(privacy_engine.sample_rate))

_model, optimizer, _train_loader = privacy_engine.make_private_with_personalization(
    module=_model,
    optimizer=optimizer,
    data_loader=_train_loader,
    noise_multiplier=noise_multiplier,
    max_grad_norm=max_grad_norm
)
results_all_reps = []

for epoch in range(1, max_epochs + 1):
    start = time.time()
    train_loss, train_acc = train(_model, device, _train_loader, optimizer, criterion, metric)
    end = time.time()
    seconds = end - start
    
    test_loss, test_acc = test(_model, device, test_loader, criterion, metric)
    
    epsilon_1 = privacy_engine.get_epsilon(0, delta)
    epsilon_2 = privacy_engine.get_epsilon(num_level1, delta)
    epsilon_3 = privacy_engine.get_epsilon(num_level1+num_level2, delta)
    
    print(f"Epoch: {epoch}")
    print(
        f"Train Loss: {train_loss:.6f} \t Acc: {100*train_acc:.2f}% "
        f"| δ: {delta} "
        f"ε1 = {epsilon_1:.4f}, "
        f"ε2 = {epsilon_2:.4f}, "
        f"ε3 = {epsilon_3:.4f}, "
    )
        
    print("Test  Loss: {:.4f} \t Acc: {:.2f}%\n".format(test_loss, 100*test_acc))
    results_all_reps.append(
        {
            "test_loss": round(test_loss,4), "test_acc": round(test_acc,4), 
             "seconds": round(seconds,4),
             "e": set(epsilon_budgets), "d": delta, "nm": round(noise_multiplier,2), "norm": max_grad_norm
        }
    )
    
    results = pd.DataFrame.from_dict(results_all_reps)
    results.to_csv("results_ours.csv", index=False)

r2 score of the curve fitting. 0.9998839695905924
0.8563 0.2948
4.2269 1.0
{0.29476189686545673, 0.556964846258241, 1.0}
Epoch: 1
Train Loss: 0.692424 	 Acc: 55.24% | δ: 0.001 ε1 = 0.0599, ε2 = 0.1257, ε3 = 0.2361, 
Test  Loss: 0.6654 	 Acc: 57.48%

Epoch: 2
Train Loss: 0.694599 	 Acc: 56.99% | δ: 0.001 ε1 = 0.0894, ε2 = 0.1868, ε3 = 0.3546, 
Test  Loss: 0.6557 	 Acc: 58.27%

Epoch: 3
Train Loss: 0.643333 	 Acc: 65.58% | δ: 0.001 ε1 = 0.1129, ε2 = 0.2357, ε3 = 0.4490, 
Test  Loss: 0.6429 	 Acc: 60.63%

Epoch: 4
Train Loss: 0.647404 	 Acc: 64.08% | δ: 0.001 ε1 = 0.1333, ε2 = 0.2778, ε3 = 0.5310, 
Test  Loss: 0.6340 	 Acc: 61.81%

Epoch: 5
Train Loss: 0.618542 	 Acc: 67.76% | δ: 0.001 ε1 = 0.1514, ε2 = 0.3153, ε3 = 0.6043, 
Test  Loss: 0.6175 	 Acc: 63.39%

Epoch: 6
Train Loss: 0.619889 	 Acc: 66.34% | δ: 0.001 ε1 = 0.1681, ε2 = 0.3500, ε3 = 0.6719, 
Test  Loss: 0.6048 	 Acc: 64.57%

Epoch: 7
Train Loss: 0.598616 	 Acc: 69.86% | δ: 0.001 ε1 = 0.1837, ε2 = 0.3822, ε3 = 0.7351, 
Test  Loss

Epoch: 64
Train Loss: 0.471411 	 Acc: 79.26% | δ: 0.001 ε1 = 0.6535, ε2 = 1.3651, ε3 = 2.7014, 
Test  Loss: 0.4427 	 Acc: 80.31%

Epoch: 65
Train Loss: 0.377675 	 Acc: 83.08% | δ: 0.001 ε1 = 0.6593, ε2 = 1.3777, ε3 = 2.7264, 
Test  Loss: 0.4432 	 Acc: 80.31%

Epoch: 66
Train Loss: 0.403498 	 Acc: 82.03% | δ: 0.001 ε1 = 0.6651, ε2 = 1.3904, ε3 = 2.7514, 
Test  Loss: 0.4429 	 Acc: 81.10%

Epoch: 67
Train Loss: 0.378754 	 Acc: 83.10% | δ: 0.001 ε1 = 0.6710, ε2 = 1.4030, ε3 = 2.7764, 
Test  Loss: 0.4442 	 Acc: 80.71%

Epoch: 68
Train Loss: 0.436187 	 Acc: 80.10% | δ: 0.001 ε1 = 0.6768, ε2 = 1.4156, ε3 = 2.8014, 
Test  Loss: 0.4438 	 Acc: 80.71%

Epoch: 69
Train Loss: 0.481578 	 Acc: 78.40% | δ: 0.001 ε1 = 0.6826, ε2 = 1.4283, ε3 = 2.8264, 
Test  Loss: 0.4437 	 Acc: 80.71%

Epoch: 70
Train Loss: 0.471566 	 Acc: 79.79% | δ: 0.001 ε1 = 0.6884, ε2 = 1.4409, ε3 = 2.8514, 
Test  Loss: 0.4439 	 Acc: 81.10%

Epoch: 71
Train Loss: 0.420274 	 Acc: 82.59% | δ: 0.001 ε1 = 0.6942, ε2 = 1.4535, ε3 = 2.8